### This notebook contains skeleton code, and discusses some ideas on how to approach the problem of designing a robot control architecuture using BLE.

# A Robot Control Class
Classes provide a means of bundling data and functionality together. It helps organize your thoughts and your code. 

Below is the skeleton of one possible class structure to deal with robot communication. You do not have to use the exact same structure, it merely serves as a reference.

In [6]:
%load_ext autoreload
%autoreload 2

from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import time
import numpy as np
import matplotlib.pyplot as plt

LOG.propagate = False

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
class RobotControl():
    # Initialize Function
    def __init__(self, ble):
        self.ble = ble
        
        # A variable to store the latest sensor value
        self.latest_tof_front_reading = None
        self.latest_tof_side_reading = None
        
        # A list to store the history of all the sensor values
        # Each item in the list is a tuple (value, time)
        # WARNING: The list could grow really fast; you need to deal with this accordingly.
        self.tof_readings = []
        self.tof2_readings = []
        
        self.imu_readings = []
        
        # A variable to store the latest imu reading
        self.latest_imu_reading = None
        
        # Activate notifications (if required)
        self.setup_notify()
    
    # A function to activate various notifications (if required)
    def setup_notify(self):
        # Code to setup various notify events
        # Ex:
        # ble.start_notify(ble.uuid['RX_TOF2'], self.tof_callback_handler)
        # ble.start_notify(ble.uuid['RX_FANCY_SENSOR'], self.fancy_sensor_callback_handler)
        self.ble.start_notify(self.ble.uuid['RX_TOF1'], self.tof_callback_handler)
        self.ble.start_notify(self.ble.uuid['RX_TOF2'], self.tof2_callback_handler)
        self.ble.start_notify(self.ble.uuid['RX_IMU'], self.imu_callback_handler)
    
    def stop_notify(self, sensor):
        self.ble.stop_notify(ble.uuid[f'RX_{sensor}'])
        
    # An example function callback handler for storing the history of the tof sensor
    # Your callback handlers should perform minimal processing!
    # Do not add a receive_* function inside the callback handler, it defeats the purpose of BLE notify
    def tof_callback_handler(self, uuid, byte_array):
        # Append a tuple (value, time) to a list
        self.tof_readings.append( ( self.ble.bytearray_to_float(byte_array), time.time() ) )
        self.update_tof_readings()
    
    def tof2_callback_handler(self, uuid, byte_array):
        # Append a tuple (value, time) to a list
        self.tof2_readings.append( ( self.ble.bytearray_to_float(byte_array), time.time() ) )
        self.update_tof2_readings()
    
    def imu_callback_handler(self, uuid, byte_array):
        self.imu_readings.append( ( self.ble.bytearray_to_float(byte_array), time.time() ) )
        self.update_imu_readings()
    
    def update_imu_readings(self):
        if len(self.imu_readings) > 10000:
            self.imu_readings = self.imu_readings[-10000:]
    
    def update_tof_readings(self):
        if len(self.tof_readings) > 10000:
            self.tof_readings = self.tof_readings[-10000:]
    
    def update_tof2_readings(self):
        if len(self.tof2_readings) > 10000:
            self.tof2_readings = self.tof2_readings[-10000:]
    
    # An example function to fetch the front TOF sensor reading
    # Here we assume RX_TOF1 is a valid UUID defined in connection.yaml and
    # in the Arduino code as well
    def get_front_tof(self):
        self.latest_tof_front_reading = self.ble.receive_float(self.ble.uuid['RX_TOF2'])
        print(self.latest_tof_front_reading)
    
    def get_side_tof(self):
        self.latest_tof_side_reading = self.ble.receive_float(self.ble.uuid['RX_TOF1'])
        print(self.latest_tof_side_reading)
    
    # An example function to fetch the IMU readings as a string
    # Here we assume RX_IMU is a valid UUID defined in connection.yaml and
    # in the Arduino code as well
    def get_imu(self):
        self.ble.send_command(CMD.GET_IMU, '')
        self.latest_imu_reading = self.ble.receive_float(self.ble.uuid['RX_IMU'])
        
        print(self.latest_imu_reading)
    
    # A function to instruct the robot to move forward
    def move_forward(self, speed, forward, doPID = 0):
        # Code to move forward
        # Ex: 
        # Here we assume the command is defined in cmd_types.py and 
        # the Artemis is programmed to handle it accordingly
        # ble.send_command(CMD.MOVE_FORWARD, speed)
        
        """
        speed: two element list ([motor1Speed, motor2Speed])
        time: duration of robot movement in seconds
        forward:
        doPID: 0 when PID should be done and 1 otherwise
        """
        self.tof_readings = []
        self.tof2_readings = []
        
        self.ble.send_command(CMD.MOVE_FORWARD, f'{speed[0]}|{speed[1]}|{forward}|{doPID}')
    
    # A function to stop robot motion
    def stop(self):
        # Code to stop robot motion
        self.ble.send_command(CMD.STOP_ROBOT, '')
    
    # A function to instruct the robot to update PID constants
    def updatePID(self, setpoint, k_p, k_i, k_d):
        self.ble.send_command(CMD.UPDATE_PID, f'{setpoint}|{k_p}|{k_i}|{k_d}')

### You can now use such a class to control your robot

In [ ]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

# Instantiate RobotControl class
rc = RobotControl(ble)

In [ ]:
startingSpeed = 75
rc.updatePID(300, 0.35, 0.00001, 0.75)

In [ ]:
rc.move_forward([startingSpeed, startingSpeed], 0)
time.sleep(10)
rc.stop()

In [ ]:
# Move backward after a test
rc.move_forward([100, 100], 1, 1) # 0 to go forward and 1 not to do PID
time.sleep(2)
rc.stop()

In [ ]:
rc.tof_readings

In [ ]:
rc.tof2_readings

In [ ]:
file = 'trial102_motor.txt'
with open(file, 'w') as f:
    for item in rc.tof_readings:
        f.write(str(item))
        f.write("\n")

In [ ]:
def plot(x):
    start_time = x[0][1]
    plt.plot([(x[i][1] - start_time) for i in range(len(x))], [x[i][0] for i in range(len(x))])
    plt.title('Front ToF Sensor Reading (mm) vs. Time (msec)')
    plt.xlim([0, 0.032])
plot(rc.tof2_readings)

In [8]:
def processRunData(file):
    with open(file, 'r') as f:
        x = f.read().splitlines()
    
    calculate = False
    comma = x[0].index(",")
    oldSensorValue = float(x[0][1:comma])
    oldTimeValue = float(x[0][comma+2:-1])

    for val in x[1:]:

        comma = val.index(",")
        newSensorValue = float(val[1:comma])
        newTimeValue = float(val[comma+2:-1])

        #print(newTimeValue-oldTimeValue)

        if (newTimeValue - oldTimeValue) > 0.01:
            timeDelta = newTimeValue - oldTimeValue
            sensorDelta = newSensorValue - oldSensorValue
            
            speed = sensorDelta/(timeDelta * 1000) # convert mm/s to m/s
            if speed > 0:
                print(f'{speed} m/s')

            oldTimeValue = newTimeValue
            oldSensorValue = newSensorValue

In [9]:
#files = ['trial60_15.txt', '300+2x.txt', '300+2x_2.txt', 'trial10.txt', 
files=[ 'trial10.txt' ]

for file in files:
    print(file)
    processRunData(file)
    print("\n")

trial10.txt
0.09931342788814435 m/s
0.13329214732894779 m/s
0.04405573265969571 m/s
0.05352289722408909 m/s
0.11443276130193981 m/s
0.1318839103229255 m/s
0.06651888582862399 m/s
0.022113225850670097 m/s
0.04435155097573742 m/s
0.06464560776184275 m/s
0.06712282554771393 m/s
0.1622943994304243 m/s
0.022472455288734584 m/s
0.04479706073972808 m/s
0.06636136953357383 m/s
0.0329847199175835 m/s
0.09774274284382646 m/s
0.1326964953769981 m/s
0.03340903587586822 m/s
0.034255994772950014 m/s
0.03345700520085511 m/s
0.03313075589363223 m/s
0.16641818167390113 m/s
0.033120421990239894 m/s
0.131035828651466 m/s
0.06641005314741415 m/s
0.0657007652002287 m/s
0.10200569089214057 m/s
0.13352553164395772 m/s
0.02201191307131574 m/s
0.02198572132470882 m/s
0.06580280983048455 m/s
0.032884637699339844 m/s
0.06655987812522315 m/s
0.04513644336830778 m/s
0.06656251884531764 m/s
0.022336860604446812 m/s
0.0221008746970176 m/s
0.032886958294455725 m/s
0.16841753599794412 m/s
0.022367235494880545 m/s
0.17

In [ ]:
!ls

In [ ]:
rc.ble.send_command(CMD.PING, "")

In [ ]:
rc.ble.receive_string(rc.ble.uuid['RX_STRING'])

In [ ]:
time.sleep(5)

speed = 80
for i in range(5):
    rc.move_forward([speed, speed], i % 2)
    time.sleep(1)
    rc.get_imu()
    rc.stop()
    time.sleep(2)
    speed += 15

In [ ]:
time.sleep(5)

for i in range(5):
    speed -= 15
    
    rc.move_forward([speed, speed], i % 2)
    time.sleep(1)
    rc.get_imu()
    rc.stop()
    time.sleep(2)

In [ ]:
rc.move_forward([50, 50], 0, 1)

In [ ]:
rc.stop()

In [ ]:
rc.setup_notify()
startTime = time.time()

while time.time() - startTime < 1:
    time.sleep(0.01)

rc.stop_notify('IMU')

In [ ]:


### Move robot forward for 3 secs and get sensor readings ###
rc.get_imu()

rc.move_forward(50)
log.info("IMU Reading: " + str(rc.latest_imu_reading))
await time.sleep(3)
rc.stop()

rc.get_imu()
log.info("IMU Reading: " + str(rc.latest_imu_reading))

<hr>

### There are two possible approaches to reading values from the Artemis board
**Approach 1:** Read values explicitly <br>
**Approach 2:** Notifications

#### You can use a combination of both i.e read some values explicitly and activate notifications for others.

## Approach 1: Read values explicitly
Below is a possible structure to run your robot commands in a loop and explicitly read the sensor values as required. 

You have more control of your code in this approach. However, the read functions could perform slower in comparison to notify events.

In [ ]:
# Add this to the top most cell containing the imports
import asyncio


while True:
    ###### Your code ######
    # Ex: Move the robot for 1 sec
    # rc.move_forward(50)
    # await asyncio.sleep(1)
    # rc.stop()
    
    ###### Read values ######
    # rc.get_imu()
    # rc.get_front_tof()

## Approach 2: Notifications
Below is a possible structure to run your robot command in a loop and utilize the notify events. 

You may have to tweak the sleep time based on how you program your arduino, system OS, system specs, system load and, the number and frequency of notify events. 

However, you **don't have to worry too much** about this if you keep your notify callback functions light. It becomes an issue only when you have a large number of notify events within a (very) short duration of time.

In [ ]:
while True:
    ###### Your code ######
    # Ex: Move the robot for 1 sec
    # rc.move_forward(50)
    # await asyncio.sleep(1)
    # rc.stop()
    
    # Process notify events
    # Sleep the current execution so that the notify callback functions can run
    # The sleep time can be much smaller, but it depends on the OS, hardware specs, number of notify events and the system load
    await asyncio.sleep(0.1)